# QRM CW2 PartB

In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
df = pd.read_csv('QRM-2021-cw2-data.csv')
# Compute Linear Loss
df['CROX_LL'] = np.log(df.CROX.shift(1)) - np.log(df.CROX)
df['NFLX_LL'] = np.log(df.NFLX.shift(1)) - np.log(df.NFLX)
df['TIF_LL'] = np.log(df.TIF.shift(1)) - np.log(df.TIF)
df['TSLA_LL'] = np.log(df.TSLA.shift(1)) - np.log(df.TSLA)
df = df.iloc[1:,:]
df.head()

,Date,CROX,NFLX,TIF,TSLA,CROX_LL,NFLX_LL,TIF_LL,TSLA_LL
1,2011-12-19,14.550,9.5929,62.43,27.75,0.019061,0.038989,0.002879,0.008969
2,2011-12-20,15.060,10.2000,63.14,27.90,-0.034451,-0.061364,-0.011309,-0.005391
3,2011-12-21,15.160,10.1386,64.65,27.57,-0.006618,0.006038,-0.023634,0.011898
4,2011-12-22,15.160,10.5486,64.57,27.77,0.000000,-0.039643,0.001238,-0.007228
5,2011-12-23,15.875,10.3743,65.18,27.90,-0.046085,0.016662,-0.009403,-0.004670


### (i) Fit a GARCH(1,1) to each stock
### $r_t = \mu + \epsilon_t, \epsilon_t = \sigma_t Z_t$, $\sigma_t ^ 2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$

In [4]:
from arch import arch_model
from arch.__future__ import reindexing
# CROX 
am_CROX = arch_model(df.CROX_LL,rescale=False,mean='Constant', lags=0, vol='Garch', p=1, o=0, q=1, power=2.0)
res_CROX = am_CROX.fit(disp='off')
res_CROX.params, res_CROX.std_err

(mu         -0.001475
 omega       0.000401
 alpha[1]    0.200001
 beta[1]     0.300000
 Name: params, dtype: float64,
 mu          0.001014
 omega       0.000147
 alpha[1]    0.256222
 beta[1]     0.372887
 Name: std_err, dtype: float64)

In [5]:
# NFLX
am_NFLX = arch_model(df.NFLX_LL,rescale=False,mean='Constant', lags=0, vol='Garch', p=1, o=0, q=1, power=2.0)
res_NFLX = am_NFLX.fit(disp="off")
res_NFLX.params, res_NFLX.std_err

(mu         -0.001308
 omega       0.000490
 alpha[1]    0.285457
 beta[1]     0.250203
 Name: params, dtype: float64,
 mu          0.000695
 omega       0.000318
 alpha[1]    0.157489
 beta[1]     0.360120
 Name: std_err, dtype: float64)

In [6]:
# TIF
am_TIF = arch_model(df.TIF_LL,rescale=False,mean='Constant', lags=0, vol='Garch', p=1, o=0, q=1, power=2.0)
res_TIF = am_TIF.fit(disp='off')
res_TIF.params, res_TIF.std_err

(mu         -0.000344
 omega       0.000102
 alpha[1]    0.100855
 beta[1]     0.599459
 Name: params, dtype: float64,
 mu          0.000406
 omega       0.000023
 alpha[1]    0.130207
 beta[1]     0.046065
 Name: std_err, dtype: float64)

In [7]:
# TSLA
am_TSLA = arch_model(df.TSLA_LL,rescale=False,mean='Constant', lags=0, vol='Garch', p=1, o=0, q=1, power=2.0)
res_TSLA = am_TSLA.fit(disp='off')
res_TSLA.params, res_TSLA.std_err

(mu         -0.000847
 omega       0.000020
 alpha[1]    0.050000
 beta[1]     0.930000
 Name: params, dtype: float64,
 mu          0.000715
 omega       0.000002
 alpha[1]    0.026125
 beta[1]     0.023911
 Name: std_err, dtype: float64)

In [8]:
dict = {'CROX': res_CROX.std_resid, 'NFLX': res_NFLX.std_resid, 'TIF': res_TIF.std_resid, 'TSLA':res_TSLA.std_resid}
std = pd.DataFrame(dict)
std.head()

,CROX,NFLX,TIF,TSLA
1,0.644683,0.953907,0.149362,0.240716
2,-1.173083,-1.605279,-0.561163,-0.114641
3,-0.175786,0.169906,-1.258108,0.331157
4,0.057267,-1.229136,0.083155,-0.170212
5,-1.819784,0.529332,-0.507341,-0.104891


In [9]:
std.to_csv('std.csv',index=False)

### （ii) Cross-correlogram and the cross-correlogram of absolute values.

In [10]:
# Absolute standardised residuals 
dict = {'CROX': abs(res_CROX.std_resid), 'NFLX': abs(res_NFLX.std_resid), 'TIF': abs(res_TIF.std_resid), 'TSLA':abs(res_TSLA.std_resid)}
ab_std = pd.DataFrame(dict)
ab_std.head()

,CROX,NFLX,TIF,TSLA
1,0.644683,0.953907,0.149362,0.240716
2,1.173083,1.605279,0.561163,0.114641
3,0.175786,0.169906,1.258108,0.331157
4,0.057267,1.229136,0.083155,0.170212
5,1.819784,0.529332,0.507341,0.104891


In [11]:
ab_std

,CROX,NFLX,TIF,TSLA
1,0.644683,0.953907,0.149362,0.240716
2,1.173083,1.605279,0.561163,0.114641
3,0.175786,0.169906,1.258108,0.331157
4,0.057267,1.229136,0.083155,0.170212
5,1.819784,0.529332,0.507341,0.104891
...,...,...,...,...
1996,0.295332,0.187034,0.453869,2.141232
1997,0.945191,0.566600,3.411517,0.276510
1998,0.398179,0.410309,0.022462,0.728026
1999,0.278525,0.362793,0.086449,0.201549


In [12]:
ab_std.to_csv('ab_std.csv',index=False)

### See R for the plot

### (iii) Fit a Gauss copula to the standardised residual using method of moments

### See R for the result

### (iv) Monte Carlo based VaR and ES forecast

In [13]:
data = pd.read_csv('stimulate.csv')
data.head()

,V1,V2,V3,V4
0,0.642831,0.820200,0.916458,0.729458
1,0.189657,0.697723,0.380910,0.848720
2,0.283927,0.508652,0.673422,0.216514
3,0.453507,0.573142,0.202932,0.901724
4,0.883367,0.757102,0.243429,0.807143


In [18]:
data #copula

,V1,V2,V3,V4
0,0.642831,0.820200,0.916458,0.729458
1,0.189657,0.697723,0.380910,0.848720
2,0.283927,0.508652,0.673422,0.216514
3,0.453507,0.573142,0.202932,0.901724
4,0.883367,0.757102,0.243429,0.807143
...,...,...,...,...
99995,0.942817,0.983372,0.518340,0.574661
99996,0.461725,0.343077,0.083224,0.710037
99997,0.302204,0.282264,0.565836,0.212246
99998,0.166292,0.012973,0.030475,0.059092


$\sigma_{t+1} ^ 2 = \omega + \alpha \epsilon_t^2 + \beta \sigma_t^2$

In [14]:
# CROX
CROX_volforcast = math.sqrt(res_CROX.params[1] + res_CROX.params[2]*(res_CROX.resid[2000]**2) + res_CROX.params[3]*(res_CROX._volatility[-1]**2))
CROX_s = [res_CROX.params[0] + CROX_volforcast*np.quantile(std.CROX,q=i) for i in data.V1]

# NFLX
NFLX_volforcast = math.sqrt(res_NFLX.params[1] + res_NFLX.params[2]*(res_NFLX.resid[2000]**2) + res_NFLX.params[3]*(res_NFLX._volatility[-1]**2))
NFLX_s = [res_NFLX.params[0] + NFLX_volforcast*np.quantile(std.NFLX,q=i) for i in data.V2]

# TIF
TIF_volforcast = math.sqrt(res_TIF.params[1] + res_TIF.params[2]*(res_TIF.resid[2000]**2) + res_TIF.params[3]*(res_TIF._volatility[-1]**2))
TIF_s = [res_TIF.params[0] + TIF_volforcast*np.quantile(std.TIF,q=i) for i in data.V3]

#TSLA
TSLA_volforcast = math.sqrt(res_TSLA.params[1] + res_TSLA.params[2]*(res_TSLA.resid[2000]**2) + res_TSLA.params[3]*(res_TSLA._volatility[-1]**2))
TSLA_s = [res_TSLA.params[0] + TSLA_volforcast*np.quantile(std.TSLA,q=i) for i in data.V4]

In [15]:
dict = {'CROX': CROX_s, 'NFLX': NFLX_s, 'TIF': TIF_s, 'TSLA':TSLA_s}
stim_return = pd.DataFrame(dict)
stim_return.head() # compute stimulated returns 

,CROX,NFLX,TIF,TSLA
0,0.005718,0.014356,0.019772,0.012200
1,-0.015645,0.008075,-0.004112,0.024281
2,-0.009665,-0.000090,0.005361,-0.019825
3,-0.002556,0.002575,-0.011510,0.032123
4,0.022124,0.010470,-0.009532,0.019152


In [16]:
# Compute the simulated portforlio losses
L = list(-stim_return.sum(axis =1)/4)
N = 100000

# Compute the 95% and 99% VaR and ES 
VaR_95 = np.quantile(L, 0.95)
VaR_99 = np.quantile(L, 0.99)

ES_95 = (1/(N*0.05))*sum([k for k in L if k > VaR_95])
ES_99 = (1/(N*0.01))*sum([k for k in L if k > VaR_99])

print(VaR_95, VaR_99,ES_95,ES_99)

0.02638111574117427 0.04607741825064669 0.0386851576865765 0.06088765086623263


In [17]:
data.head()

,V1,V2,V3,V4
0,0.642831,0.820200,0.916458,0.729458
1,0.189657,0.697723,0.380910,0.848720
2,0.283927,0.508652,0.673422,0.216514
3,0.453507,0.573142,0.202932,0.901724
4,0.883367,0.757102,0.243429,0.807143
